In [1]:
using SymPy

In [2]:
function _sym(x::Expr)
    y = x
    y.args .= _sym.(y.args)
    return  y
end

_sym(x::Number) = Sym(x)
_sym(x) = x

macro sym(x)
    return esc(_sym(x))
end

@sym (macro with 1 method)

In [3]:
function get_glrk_nodes(s)
    if s == 1
        c = @sym Vector([1//2])
    elseif s == 2
        c = @sym [1/2-√3/6, 1/2+√3/6]
    elseif s == 3
        c = @sym [1/2-√15/10,  1/2,        1/2+√15/10 ]
    elseif s == 4
        c = @sym [
          1/2-√(3/7+2*√30/35)/2,
          1/2-√(3/7-2*√30/35)/2,
          1/2+√(3/7-2*√30/35)/2,
          1/2+√(3/7+2*√30/35)/2
        ]
    else
        @error "GLRK nodes for " * string(s) * " stages not implemented."
    end

    return c
end

get_glrk_nodes (generic function with 1 method)

In [4]:
function get_lobatto_nodes(s)
    if s == 2
        c = @sym [0, 1]
    elseif s == 3
        c = @sym [0, 1//2, 1]
    elseif s == 4
        c = @sym [0,  (5-√5)/10,   (5+√5)/10,  1]
    elseif s == 5
        c = @sym [0,  1//2-√21/14,  1//2,  1//2+√21/14,  1]
    else
        @error "Lobatto nodes for " * string(s) * " stages not implemented."
    end

    return c
end

get_lobatto_nodes (generic function with 1 method)

In [5]:
function get_glrk_lobatto_tableau(s)
    c = get_glrk_nodes(s)
    c̄ = get_lobatto_nodes(s+1)
    a = [Sym("a_" * string(i) * string(j)) for i in 0:s, j in 1:s]
    
    eqs = []
    for i in 0:s, k in 1:s
        eq = - c̄[i+1]^k / k
        for j in 1:s
            eq += a[i+1,j] * c[j]^(k-1)
        end
        push!(eqs, eq)
    end
    
    eqs = Sym.(eqs)
    sol = solve(eqs, a[:])
    
    A = [sol[a[i,j]] for i in axes(a,1), j in axes(a,2)]
end

get_glrk_lobatto_tableau (generic function with 1 method)

In [6]:
get_glrk_lobatto_tableau(1)

2×1 Array{Sym,2}:
 0
 1

In [7]:
get_glrk_lobatto_tableau(2)

3×2 Array{Sym,2}:
               0                 0
 sqrt(3)/8 + 1/4  -sqrt(3)/8 + 1/4
             1/2               1/2

In [8]:
get_glrk_lobatto_tableau(3)

4×3 Array{Sym,2}:
                                 0  …                                  0
 -sqrt(5)/180 + sqrt(15)/30 + 5/36     -sqrt(15)/30 - sqrt(5)/180 + 5/36
  sqrt(5)/180 + sqrt(15)/30 + 5/36     -sqrt(15)/30 + sqrt(5)/180 + 5/36
                              5/18                                  5/18

In [9]:
#get_glrk_lobatto_tableau(4)